# Limpieza de datos

## Lectura de datos

In [15]:
import pandas as pd

Lectura de datos y selección de variables

In [16]:
%%time
df = pd.read_parquet('../movilidad_mcd/data_google/GoogleCloud.parquet')
df = df[['id_adv', 'lat', 'lon', 'timestamp']]

CPU times: user 1min 6s, sys: 56.7 s, total: 2min 3s
Wall time: 4min 15s


Dimensión del data frame

In [17]:
%%time
df.shape

CPU times: user 28 µs, sys: 185 µs, total: 213 µs
Wall time: 215 µs


(80582452, 4)

Resumen de pings por dispositivo

In [18]:
%%time
num_ping_0 = pd.DataFrame(df['id_adv'].value_counts())
num_ping_0.set_axis(['freq_0'], axis='columns', inplace=True)
num_ping_0['id_adv'] = list(num_ping_0.index)
num_ping_0.reset_index(drop=True, inplace=True)
num_ping_0 = num_ping_0[['id_adv', 'freq_0']]

CPU times: user 6.52 s, sys: 406 ms, total: 6.93 s
Wall time: 7.37 s


Muestra de los datos

In [19]:
%%time
num_ping_0['freq_0'].head()

CPU times: user 215 µs, sys: 195 µs, total: 410 µs
Wall time: 2.41 ms


0    18297
1    17639
2    17343
3    16995
4    16552
Name: freq_0, dtype: int64

Cantidad de registros previo a limpieza:

In [20]:
%%time
df.shape[0]

CPU times: user 13 µs, sys: 0 ns, total: 13 µs
Wall time: 14.8 µs


80582452

## Paso 1: Se eliminan registros duplicados

Se eliminan datos duplicados

In [22]:
%%time
df = df.drop_duplicates()

CPU times: user 2min 8s, sys: 4min 22s, total: 6min 31s
Wall time: 8min 55s


Resumen de pings por dispositivo

In [23]:
%%time
num_ping_1 = pd.DataFrame(df['id_adv'].value_counts())
num_ping_1.set_axis(['freq_1'], axis='columns', inplace=True)
num_ping_1['id_adv'] = list(num_ping_1.index)
num_ping_1.reset_index(drop=True, inplace=True)
num_ping_1 = num_ping_1[['id_adv', 'freq_1']]

CPU times: user 6.82 s, sys: 467 ms, total: 7.29 s
Wall time: 8.1 s


In [24]:
%%time
num_ping_1['freq_1'].head()

CPU times: user 255 µs, sys: 229 µs, total: 484 µs
Wall time: 1.21 ms


0    17500
1    16959
2    16780
3    16297
4    15982
Name: freq_1, dtype: int64

Registros eliminados

In [25]:
%%time
sum(num_ping_0.freq_0) -sum(num_ping_1.freq_1)

CPU times: user 41.2 ms, sys: 1.82 ms, total: 43 ms
Wall time: 43.4 ms


928070

Cantidad de registros:

In [26]:
%%time
df.shape[0]

CPU times: user 22 µs, sys: 22 µs, total: 44 µs
Wall time: 46.3 µs


79654382

In [27]:
%%time
tracking_num_ping = pd.merge(num_ping_0, num_ping_1, on='id_adv', how='left')

CPU times: user 203 ms, sys: 21.4 ms, total: 224 ms
Wall time: 258 ms


## Paso 2: Se eliminan ubicaciones con menos de 7 decimales

In [28]:
from math import log10

def num_decimal(vector):
    vector = abs(vector)
    dec_vector = []
    for item in vector:
        #print(v)
        dec_vector.append(int(log10(float(str(item)[::-1])))+1)
    return (dec_vector)

Calculando la cantidad de decimales

In [29]:
%%time
(dec_lat, dec_lon) = (num_decimal(df.lat), num_decimal(df.lon))

CPU times: user 1min 54s, sys: 1.94 s, total: 1min 56s
Wall time: 1min 58s


In [35]:
%%time
df.insert(df.shape[1], 'dec_lat', dec_lat, allow_duplicates=False)
df.insert(df.shape[1], 'dec_lon', dec_lon, allow_duplicates=False)

CPU times: user 13.4 s, sys: 1.71 s, total: 15.1 s
Wall time: 16.1 s


In [37]:
%%time
df[['dec_lat', 'dec_lon']].head()

CPU times: user 651 ms, sys: 1.44 s, total: 2.09 s
Wall time: 2.88 s


,dec_lat,dec_lon
0,6,5
1,6,5
2,13,12
3,12,12
4,13,12


Resumen de cantidad de decimales

In [39]:
%%time
print("6 decimales o menos en latitud y longitud ----------")
print(df[(df.dec_lat <= 6) & (df.dec_lon <= 6)].shape)

print("6 decimales o menos en latitud o longitud ----------")
print(df[(df.dec_lat <= 6) & (df.dec_lon > 6)].shape)
print(df[(df.dec_lat > 6) & (df.dec_lon <= 6)].shape)

print("Más de 6 decimales en latitud y longitud -----------")
print(df[(df.dec_lat > 6) & (df.dec_lon > 6)].shape)

6 decimales o menos en latitud y longitud ----------
(32041700, 6)
6 decimales o menos en latitud o longitud ----------
(9045, 6)
(9207, 6)
Más de 6 decimales en latitud y longitud -----------
(47594430, 6)
CPU times: user 5.13 s, sys: 7.56 s, total: 12.7 s
Wall time: 18.9 s


El siguiente paso es conservar únicamente los registros con más de 6 decimales en latitud y longitud.

In [41]:
%%time
df = df[(df.dec_lat > 6) & (df.dec_lon > 6)]
df[['dec_lat', 'dec_lon']].head()

CPU times: user 2.54 s, sys: 3.55 s, total: 6.08 s
Wall time: 15.7 s


,dec_lat,dec_lon
2,13,12
3,12,12
4,13,12
7,13,12
11,13,11


Resumen de pings por dispositivo

In [43]:
%%time
num_ping_2 = pd.DataFrame(df['id_adv'].value_counts())
num_ping_2.set_axis(['freq_2'], axis='columns', inplace=True)
num_ping_2['id_adv'] = list(num_ping_2.index)
num_ping_2.reset_index(drop=True, inplace=True)
num_ping_2 = num_ping_2[['id_adv', 'freq_2']]

CPU times: user 5.24 s, sys: 389 ms, total: 5.63 s
Wall time: 6.2 s


In [44]:
num_ping_2['freq_2'].head()

0    10405
1     9654
2     6476
3     5781
4     5422
Name: freq_2, dtype: int64

Cantidad de registros:

In [45]:
%%time
df.shape[0]

CPU times: user 21 µs, sys: 11 µs, total: 32 µs
Wall time: 34.8 µs


47594430

In [46]:
%%time
tracking_num_ping = pd.merge(tracking_num_ping, num_ping_2, on='id_adv', how='left')

CPU times: user 211 ms, sys: 41.4 ms, total: 252 ms
Wall time: 310 ms


## Paso 3: Se eliminan dispositivos con un ping en el periodo completo

In [47]:
%%time
id_filtro3 = num_ping_2[(num_ping_2.freq_2 == 1)]
len(id_filtro3)

CPU times: user 2.93 ms, sys: 3.71 ms, total: 6.64 ms
Wall time: 8.61 ms


914

In [48]:
%%time
df = df[~df.id_adv.isin(id_filtro3.id_adv)]

CPU times: user 4.98 s, sys: 3.61 s, total: 8.59 s
Wall time: 15.4 s


In [49]:
%%time
df = df.drop(['dec_lat', 'dec_lon'], axis=1)

CPU times: user 1.64 s, sys: 1.85 s, total: 3.49 s
Wall time: 4.85 s


Resumen de pings por dispositivo

In [50]:
%%time
num_ping_3 = pd.DataFrame(df['id_adv'].value_counts())
num_ping_3.set_axis(['freq_3'], axis='columns', inplace=True)
num_ping_3['id_adv'] = list(num_ping_3.index)
num_ping_3.reset_index(drop=True, inplace=True)
num_ping_3 = num_ping_3[['id_adv', 'freq_3']]

CPU times: user 4.95 s, sys: 246 ms, total: 5.19 s
Wall time: 5.26 s


In [51]:
num_ping_3['freq_3'].head()

0    10405
1     9654
2     6476
3     5781
4     5422
Name: freq_3, dtype: int64

Cantidad de registros

In [52]:
%%time
df.shape[0]

CPU times: user 32 µs, sys: 235 µs, total: 267 µs
Wall time: 275 µs


47593516

In [53]:
%%time
tracking_num_ping = pd.merge(tracking_num_ping, num_ping_3, on='id_adv', how='left')

CPU times: user 228 ms, sys: 47.6 ms, total: 276 ms
Wall time: 303 ms


## Ordenando datos por timestamp e id_adv

In [54]:
df_ord = df.sort_values(['id_adv', 'timestamp'])

In [55]:
print(df_ord.shape)
print(df.shape)

(47593516, 4)
(47593516, 4)


## Paso 4: Eliminar registros con misma hora y diferente ubicación

Registros con duplicado

In [57]:
misma_hora_dif_lugar = df_ord.duplicated(subset=['id_adv', 'timestamp'], keep = False)
misma_hora_dif_lugar.shape

(47593516,)

In [58]:
df = df_ord.drop_duplicates(subset=['id_adv', 'timestamp'], keep = 'last')

In [59]:
df.shape

(47592840, 4)

Resumen de pings por dispositivo

In [60]:
%%time
num_ping_4 = pd.DataFrame(df['id_adv'].value_counts())
num_ping_4.set_axis(['freq_4'], axis='columns', inplace=True)
num_ping_4['id_adv'] = list(num_ping_4.index)
num_ping_4.reset_index(drop=True, inplace=True)
num_ping_4 = num_ping_4[['id_adv', 'freq_4']]

CPU times: user 1.88 s, sys: 183 ms, total: 2.07 s
Wall time: 2.22 s


In [63]:
num_ping_4['freq_4'].head()

0    10403
1     9649
2     6475
3     5779
4     5419
Name: freq_4, dtype: int64

In [61]:
%%time
tracking_num_ping = pd.merge(tracking_num_ping, num_ping_4, on='id_adv', how='left')

CPU times: user 214 ms, sys: 54.9 ms, total: 269 ms
Wall time: 329 ms


In [62]:
df.shape

(47592840, 4)

## Resumen de Cambios

In [67]:
tracking_num_ping[['freq_0', 'freq_1', 'freq_2', 'freq_3', 'freq_4']].head()

,freq_0,freq_1,freq_2,freq_3,freq_4
0,18297,17500,511.0,511.0,511.0
1,17639,16959,2890.0,2890.0,2890.0
2,17343,16780,1233.0,1233.0,1233.0
3,16995,16297,1937.0,1937.0,1937.0
4,16552,15982,2296.0,2296.0,2296.0


In [68]:
tracking_num_ping.describe()

,freq_0,freq_1,freq_2,freq_3,freq_4
count,306963.000000,306963.000000,289214.000000,288300.000000,288300.000000
mean,262.515196,259.491802,164.564751,165.083302,165.080957
std,757.558001,736.451915,215.960734,216.106024,216.100323
min,1.000000,1.000000,1.000000,2.000000,2.000000
25%,53.000000,53.000000,55.000000,55.000000,55.000000
50%,118.000000,117.000000,112.000000,113.000000,113.000000
75%,216.000000,215.000000,191.000000,191.000000,191.000000
max,18297.000000,17500.000000,10405.000000,10405.000000,10403.000000
